In [44]:
%run 'dataframes.ipynb'

In [45]:
# make a copy of each dataframe
copy_products_df=products_df.copy()
copy_orders_df=orders_df.copy()
copy_orderlines_df=orderlines_df.copy()
copy_brands_df=brands_df.copy()

### Data cleaning
#### Products

##### remove duplicated rows

In [46]:
#calculate the count of duplicated rows
copy_products_df.duplicated().sum()

np.int64(8746)

In [47]:
#shows the duplicated rows
products_duplicated_mask=copy_products_df.duplicated()
copy_products_df.loc[products_duplicated_mask]

,sku,name,desc,price,promo_price,in_stock,type
101,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
102,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
103,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
104,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
105,APP0390,"Apple MacBook Pro 133 ""Core i5 25GHz | 4GB RAM...",MacBook Pro laptop 133 inches (MD101Y / A).,1199,11.455.917,0,1282
...,...,...,...,...,...,...,...
16831,APP2302,"Apple MacBook Pro 13 ""Core i5 Touch Bar 33GHz ...",New MacBook Pro 13-inch Core i5 Touch Bar 33 G...,26.155.941,26.155.941,0,"1,02E+12"
16833,APP2303,"Apple MacBook Pro 13 ""Core i5 Touch Bar 33GHz ...",New MacBook Pro 13 inch Touch Bar 33 GHz Core ...,237.559.421,23.755.942,0,"1,02E+12"
18190,PAR0077,Parrot Bebop Drone 2 Power,Drone cuadricóptero quality camera integrated ...,699.9,6.733.892,0,11905404
18308,NKI0010,Nokia Wireless sphygmomanometer Plata,Sphygmomanometer for iPhone iPad and iPod App.,129.99,1.149.899,1,11905404


In [48]:
#drop duplicated rows
copy_products_df.drop_duplicates(inplace=True)

In [49]:
#check again if duplicated have been drop
copy_products_df.duplicated().sum()

np.int64(0)

In [50]:
# we reset index so that starts with 0 with step=1
copy_products_df=copy_products_df.reset_index(drop=True)

##### sku column cleaning

In [51]:
#sku must be unique,we check if there is any duplicated sku
copy_products_df['sku'].duplicated().sum()

np.int64(1)

In [52]:
#which sku is duplicated
sku_duplicated_mask=copy_products_df['sku'].duplicated()
copy_products_df.loc[sku_duplicated_mask]

,sku,name,desc,price,promo_price,in_stock,type
2805,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,NaN,1305.59,0,1282


In [53]:
#see the duplicated sku rows
copy_products_df.loc[copy_products_df['sku']=='APP1197']

,sku,name,desc,price,promo_price,in_stock,type
2804,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282
2805,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,NaN,1305.59,0,1282


In [54]:
#drop one of the duplicated sku rows
copy_products_df.drop(2805,inplace=True)

In [55]:
copy_products_df['sku'].duplicated().sum()

np.int64(0)

In [56]:
#check if duplicated sku is droped
copy_products_df['sku'].duplicated().sum()

np.int64(0)

In [57]:
#check where we have null values in products
copy_products_df.isnull().sum()

sku             0
name            0
desc            7
price          45
promo_price     0
in_stock        0
type           50
dtype: int64

In [58]:
#we can replace null descriptions with unknown
copy_products_df.loc[copy_products_df['desc'].isnull(),'desc']='unknown'

In [59]:
#because price is an important column, we need to drop those rows with null prices
copy_products_df.dropna(subset='price',inplace=True)

In [60]:
# we can replace null types with 0
copy_products_df.loc[copy_products_df['type'].isnull(),'type']=0

In [61]:
#we dont have any null values in products data frame anymore
copy_products_df.isnull().sum()

sku            0
name           0
desc           0
price          0
promo_price    0
in_stock       0
type           0
dtype: int64

In [62]:
# price and promo_price needs to be converted into float type 

In [63]:
#if you try to convert them you get an error
# pd.to_numeric(copy_products_df['price'])
# pd.to_numeric(copy_products_df['promo_price'])
# It seems that we have some unsual prices with two periods in them

In [64]:
#lets see how many unusual prices do we have
(copy_products_df['price'].str.count(r'\.')>1).value_counts(normalize=True)

price
False    0.964211
True     0.035789
Name: proportion, dtype: float64

In [65]:
#lets see how many unusual promotion prices do we have
(copy_products_df['promo_price'].str.count(r'\.')>1).value_counts(normalize=True).reset_index()

,promo_price,proportion
0,False,0.563604
1,True,0.436396


In [66]:
#lets see how many unusula price and promotion price do we have
((copy_products_df['promo_price'].str.count(r'\.')>1) & (copy_products_df['price'].str.count(r'\.')>1)).value_counts(normalize=True)

False    0.973799
True     0.026201
Name: proportion, dtype: float64

In [67]:
# As around 43 percent of promotion prices are corrupted, we just have to drop the column
# This is an important column but we have a dynamic unit_price in orderlines which can represent the promotion price very good
copy_products_df.drop('promo_price',axis=1,inplace=True)

In [68]:
#lets take a look of those prices that are corrupted to maybe find some correlation to fix them
#we make a numberic_price that its value is either price for not corrputed values and null for corrupted ones
copy_products_df['numeric_price']=pd.to_numeric(copy_products_df['price'],errors='coerce')
corrupted_price_copy_products_df=copy_products_df.loc[copy_products_df['numeric_price'].isnull(),]
# to look for a correlation we need to take a look at promotion price for these skus, so we merge this df with orderlines
corrupted_price_products_orderlines_merged=corrupted_price_copy_products_df.merge(copy_orderlines_df,on='sku',how='left')
corrupted_price_products_orderlines_merged.loc[:,['price','unit_price']].sample(50)
# It seems that we do not have any correlatin and the only way is to drop the small porion of corrupted prices

,price,unit_price
138,1.639.792,128.99
2196,42.917.885,3.637.90
4251,2.499.013,24.99
3111,115.900.092,1.159.00
528,1.639.792,160.19
3941,13.290.011,1.329.00
3477,115.900.092,1.159.00
3731,13.290.011,1.329.00
514,1.639.792,158.99
4133,13.290.011,1.329.00


In [69]:
# we drop the rows with unusual prices
copy_products_df=copy_products_df[copy_products_df['price'].str.count(r'\.')==1]

In [70]:
# now we can convert prices to numeric
copy_products_df.loc[:,'price']=pd.to_numeric(copy_products_df['price'])

In [71]:
#and also we dont need numeric price anymore
copy_products_df.drop('numeric_price',axis=1,inplace=True)

In [72]:
#check if now the tyes are correct
copy_products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6942 entries, 0 to 10579
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sku       6942 non-null   object
 1   name      6942 non-null   object
 2   desc      6942 non-null   object
 3   price     6942 non-null   object
 4   in_stock  6942 non-null   int64 
 5   type      6942 non-null   object
dtypes: int64(1), object(5)
memory usage: 379.6+ KB


In [73]:
# we need to reset indexes after the cleaning is finished 
copy_products_df.reset_index(drop=True,inplace=True)

In [74]:
copy_products_df.to_csv('../data/clean')

IsADirectoryError: [Errno 21] Is a directory: '../data/clean'